##### Simulationen für MCAR mit den Top5 erklärenden Variablen (standardisiert)
In dem Notebook 'ols_prep' wird eine lineare Regression des Laptop-Preises auf alle Exogenen durchgeführt.
Anschließend werden alle Betas standardisiert und absteigend nach ihrer (absoluten) Größe sortiert. Daraus resultiert der Erklärungsgehalt der verschiedenen Exogenen. Wir nehmen die fünf Exogenen, die den größten Erklärungsgehalt haben und standardisieren sie. Sie dienen als Grundlage für die Simulationen in diesem Notebook. 
Es werden MCAR-Simulationen mit k=1000 durchgeführt, wobei jeweils nur die Top5 standardisierten Exogenen verwendet werden.
Es wird kein Jitter hinzugefügt.

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

rng = np.random.RandomState(0)

In [7]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('../Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index

# Nur die 5 Spalten auswählen, die auch den größten Erklärungsgehalt haben
selected_columns = ['Preis', 'SSD', 'Mobilfunk_vorhanden', 'PPI_Display', 'Takt_Prozessor', 'Marke_Lenovo']

# Create a new DataFrame with only the selected columns
df = df[selected_columns]

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])
imputed_stats_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'], index = np.arange(0.1, 1, 0.1))

no_imputation = pd.DataFrame(columns = ['SE', 'Alpha'])

# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)

# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

# Wahrer Mittelwert:
true_mean = df['Preis'].mean()

In [8]:
def contains_mean(values):
    return 1 if (true_mean > (np.mean(values) - 1.95996 * stats.sem(values))) and (true_mean < (np.mean(values) + 1.95996 * stats.sem(values))) else 0

In [9]:
# OLS Model 
# fitting the model 
# df_noPrice ist bereits standardisiert
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     262.7
Date:                Sat, 23 Mar 2024   Prob (F-statistic):          3.90e-181
Time:                        22:16:09   Log-Likelihood:                -7915.8
No. Observations:                1038   AIC:                         1.584e+04
Df Residuals:                    1032   BIC:                         1.587e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1333.9633    

In [10]:
print(df.corr().to_latex(escape=False, float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  Preis &  SSD &  Mobilfunk_vorhanden &  PPI_Display &  Takt_Prozessor &  Marke_Lenovo \\
\midrule
Preis               &   1.00 & 0.62 &                 0.18 &         0.52 &            0.36 &          0.03 \\
SSD                 &   0.62 & 1.00 &                 0.11 &         0.43 &            0.20 &          0.05 \\
Mobilfunk_vorhanden &   0.18 & 0.11 &                 1.00 &         0.07 &           -0.20 &          0.35 \\
PPI_Display         &   0.52 & 0.43 &                 0.07 &         1.00 &            0.00 &         -0.02 \\
Takt_Prozessor      &   0.36 & 0.20 &                -0.20 &         0.00 &            1.00 &         -0.15 \\
Marke_Lenovo        &   0.03 & 0.05 &                 0.35 &        -0.02 &           -0.15 &          1.00 \\
\bottomrule
\end{tabular}



C:\Users\Admin\AppData\Local\Temp\ipykernel_17644\4022997472.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.corr().to_latex(escape=False, float_format="%.2f"))


In [11]:
def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

In [12]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    return [np.mean(list(train_labels) + imputed_values), np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values), contains_mean(list(train_labels) + imputed_values)]

In [13]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)

    mean_knn_1 = np.mean(list(train_labels.values)+imputed_values_knn_1)
    mean_knn_3 = np.mean(list(train_labels.values)+imputed_values_knn_3)
    mean_knn_5 = np.mean(list(train_labels.values)+imputed_values_knn_5)

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)

    alpha_knn_1 = contains_mean(list(train_labels.values)+imputed_values_knn_1)
    alpha_knn_3 = contains_mean(list(train_labels.values)+imputed_values_knn_3)
    alpha_knn_5 = contains_mean(list(train_labels.values)+imputed_values_knn_5)


    return [mean_knn_1, mse_knn_1, sem_knn_1, alpha_knn_1, mean_knn_3, mse_knn_3, sem_knn_3, alpha_knn_3, mean_knn_5, mse_knn_5, sem_knn_5, alpha_knn_5]

In [14]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        return_values = []
        
        for column in imputed_simul_knn.columns:
            column_mean = np.mean(imputed_simul_knn[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_knn.loc[c] =  return_values
    imputed_stats_knn

simul_knn()
imputed_stats_knn

100%|██████████| 1000/1000 [01:53<00:00,  8.80it/s]


,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1328.565937,238540.96343,23.234547,1.0,1331.988477,185534.511638,23.0386,1.0,1331.205839,182299.996821,22.980003,1.0
0.2,1324.60376,247473.606779,23.275014,1.0,1331.683844,191127.825881,22.838787,1.0,1329.994028,186953.860968,22.699361,1.0
0.3,1322.76872,252292.243718,23.277823,0.997,1333.128707,195965.461682,22.584549,1.0,1331.050219,190441.270356,22.349102,1.0
0.4,1322.349781,264128.775396,23.337573,0.981,1335.248493,201707.525254,22.343164,0.998,1332.804418,195806.328021,21.968541,1.0
0.5,1325.32738,270282.004365,23.294518,0.958,1338.461056,208016.489519,22.040703,0.993,1336.299758,202917.094151,21.548843,0.993
0.6,1328.91793,282223.496581,23.223015,0.926,1342.051068,217141.292209,21.618068,0.962,1339.646165,215071.856982,20.988811,0.964
0.7,1336.392979,300638.450959,23.173543,0.841,1347.978237,234263.6786,21.119442,0.837,1344.154123,233663.104516,20.336278,0.869
0.8,1343.045233,329708.302285,23.075595,0.75,1347.312836,259524.500298,20.389801,0.735,1338.675537,257045.185237,19.309978,0.736
0.9,1348.468122,384464.182561,22.751517,0.54,1331.684444,295682.69118,19.056415,0.553,1313.375049,286608.34969,17.512624,0.491


In [15]:
def simul_ols():
    
    for c in np.arange(0.1, 1, 0.1):
        
        imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])

        return_values = []

        for column in imputed_simul_ols.columns:
            column_mean = np.mean(imputed_simul_ols[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_ols.loc[c] =  return_values
    imputed_stats_ols

simul_ols()

100%|██████████| 1000/1000 [00:07<00:00, 126.29it/s]


In [16]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1).to_csv('MCAR_Simulation_Top5-10 (neu)')
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,Mean,MSE OLS,SE OLS,Alpha OLS,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1333.738312,249659.110343,22.713,1.0,1328.565937,238540.96343,23.234547,1.0,1331.988477,185534.511638,23.0386,1.0,1331.205839,182299.996821,22.980003,1.0
0.2,1334.110696,251713.287248,22.193939,1.0,1324.60376,247473.606779,23.275014,1.0,1331.683844,191127.825881,22.838787,1.0,1329.994028,186953.860968,22.699361,1.0
0.3,1333.850883,251872.930957,21.636826,1.0,1322.76872,252292.243718,23.277823,0.997,1333.128707,195965.461682,22.584549,1.0,1331.050219,190441.270356,22.349102,1.0
0.4,1334.32126,250824.794466,21.109029,1.0,1322.349781,264128.775396,23.337573,0.981,1335.248493,201707.525254,22.343164,0.998,1332.804418,195806.328021,21.968541,1.0
0.5,1333.363174,251722.150409,20.546581,0.994,1325.32738,270282.004365,23.294518,0.958,1338.461056,208016.489519,22.040703,0.993,1336.299758,202917.094151,21.548843,0.993
0.6,1335.002161,252452.892121,19.997664,0.968,1328.91793,282223.496581,23.223015,0.926,1342.051068,217141.292209,21.618068,0.962,1339.646165,215071.856982,20.988811,0.964
0.7,1335.263173,254255.544448,19.465585,0.895,1336.392979,300638.450959,23.173543,0.841,1347.978237,234263.6786,21.119442,0.837,1344.154123,233663.104516,20.336278,0.869
0.8,1334.554382,257761.167141,18.948179,0.756,1343.045233,329708.302285,23.075595,0.75,1347.312836,259524.500298,20.389801,0.735,1338.675537,257045.185237,19.309978,0.736
0.9,1335.714673,268481.32803,18.565886,0.509,1348.468122,384464.182561,22.751517,0.54,1331.684444,295682.69118,19.056415,0.553,1313.375049,286608.34969,17.512624,0.491
